<a href="https://colab.research.google.com/github/nsk-ai/RAG-Bootcamp-2025/blob/main/Chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://python.langchain.com/docs/concepts/text_splitters/

---
## Text Splitters (Chunking)

In [ ]:
pip install langchain

In [ ]:
pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os

# Access the secret using userdata.get()
my_variable = userdata.get('GROQ_API_KEY')

# You can also set it as an environment variable for use with os.getenv()
os.environ['GROQ_API_KEY'] = my_variable

In [ ]:
%pip install -qU langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [ ]:
%pip install -qU langchain-text-splitters

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./sample_data/langchain.txt");
docs = loader.load();

In [ ]:
import pprint

pprint.pp(docs[0].metadata)

{'source': './sample_data/langchain.txt'}


In [ ]:
print(len(docs))

1


In [ ]:
docs[0].page_content[:100]

'LangChain: The Definitive Framework for Building Advanced Language Model Applications\n\nLangChain has'

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([docs[0].page_content])
print(texts[0])

page_content='LangChain: The Definitive Framework for Building Advanced Language Model Applications'


In [ ]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document

# Example of a long document content
long_content = """
This is the first paragraph. It is somewhat long, but might not be the primary cause of issues.

This is the second paragraph. This paragraph is intentionally made to be much, much longer than your specified chunk_size of 200 characters. It will continue on and on, demonstrating what happens when a natural split point (like a double newline) occurs, but the content between those natural splits is excessively large. This is a common scenario with articles, technical documentation, or prose where paragraphs can span hundreds or even thousands of characters. The CharacterTextSplitter's main job is to cut at the designated separator. If it finds a large block of text that doesn't contain that specific separator within it, it will keep that entire block as a single chunk, even if it vastly exceeds the desired chunk_size. This behavior is by design for this specific splitter, as its name implies, it's about splitting by *characters* (the separator), not strictly by *size*. Other splitters like RecursiveCharacterTextSplitter are designed to be more aggressive in breaking down chunks to meet size constraints, using a hierarchy of separators. Understanding this distinction is key to choosing the right tool for your text splitting needs in LangChain. This lengthy paragraph will undoubtedly cause the warning to appear several times, as it represents a single, indivisible unit according to the "\n\n" separator.

This is the third paragraph, which is also quite long, to further illustrate the point. It ensures that there are multiple instances where the splitter will be forced to create chunks larger than the desired size because the content between the defined separators is extensive. This helps solidify the understanding that while chunk_size is a guideline, the separator acts as a hard boundary that the CharacterTextSplitter respects above all else.
"""

# Simulate a Document object
docs = [Document(page_content=long_content, metadata={"source": "example"})]

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([docs[0].page_content])

print("--- Chunks created ---")
for i, text_doc in enumerate(texts):
    print(f"Chunk {i+1} (length: {len(text_doc.page_content)}):")
    print(text_doc.page_content[:100] + "...") # Print first 100 chars for brevity
    print("-" * 30)

--- Chunks created ---
Chunk 1 (length: 95):
This is the first paragraph. It is somewhat long, but might not be the primary cause of issues....
------------------------------
Chunk 2 (length: 1309):
This is the second paragraph. This paragraph is intentionally made to be much, much longer than your...
------------------------------
Chunk 3 (length: 12):
" separator....
------------------------------
Chunk 4 (length: 447):
This is the third paragraph, which is also quite long, to further illustrate the point. It ensures t...
------------------------------


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
)
recursive_texts = recursive_splitter.create_documents([docs[0].page_content])

for i, text_doc in enumerate(recursive_texts):
    print(f"Recursive Chunk {i+1} (length: {len(text_doc.page_content)}):")
    print(text_doc.page_content[:100] + "...")
    print("-" * 30)

Recursive Chunk 1 (length: 95):
This is the first paragraph. It is somewhat long, but might not be the primary cause of issues....
------------------------------
Recursive Chunk 2 (length: 195):
This is the second paragraph. This paragraph is intentionally made to be much, much longer than your...
------------------------------
Recursive Chunk 3 (length: 189):
what happens when a natural split point (like a double newline) occurs, but the content between thos...
------------------------------
Recursive Chunk 4 (length: 199):
articles, technical documentation, or prose where paragraphs can span hundreds or even thousands of ...
------------------------------
Recursive Chunk 5 (length: 194):
If it finds a large block of text that doesn't contain that specific separator within it, it will ke...
------------------------------
Recursive Chunk 6 (length: 186):
chunk_size. This behavior is by design for this specific splitter, as its name implies, it's about s...
-----------------------------

#### When to Use `CharacterTextSplitter` and Its Alternatives

**`CharacterTextSplitter`** is like a simple pair of scissors for your text. You tell it exactly where to cut (e.g., at every double newline `"\n\n"` for paragraphs, or every comma `,` in a list).

**Use `CharacterTextSplitter` when:**

* **You have very specific, clear breaking points:** Imagine a document where `"\n\n"` always means a new, self-contained section, and you absolutely *do not* want to break those sections.
* **Working with structured data:** Like processing a simple CSV where each line is a record, or log files where each entry is separated by a unique character sequence.
* **Small documents or less strict size needs:** If the document is small, or you're okay with chunks being much larger than your `chunk_size` if a natural break point doesn't appear soon enough.
    * **_Important Note:_** If a piece of text *between* your chosen separators is very long, `CharacterTextSplitter` will keep it as one big chunk, even if it goes way over your desired `chunk_size`. It prioritizes the separator over the size limit.

---

#### When NOT to Use `CharacterTextSplitter` (and What to Use Instead)

For most situations, especially when preparing text for **Large Language Models (LLMs)** like in **Retrieval-Augmented Generation (RAG)**, `CharacterTextSplitter` often isn't the best fit because of its strict reliance on a single separator.

**Instead, consider these for better results:**

1.  **`RecursiveCharacterTextSplitter` (Your Go-To Choice!):**
    * **Why it's better:** Think of this as a smart pair of scissors with multiple blades. It tries to cut using a list of separators (e.g., first by `"\n\n"`, then by `"\n"`, then by spaces `" "`, then by individual characters `""`).
    * **Niche Use:** **General-purpose text splitting.** This is the recommended splitter for most documents (articles, books, reports) because it tries harder to keep chunks close to your `chunk_size` while still respecting natural breaks in the text.

2.  **`MarkdownHeaderTextSplitter`:**
    * **Niche Use:** **Markdown documents.** If your text is in Markdown format (using `#`, `##`, `###` for headers), this splitter understands that structure and will split your document based on these headings, keeping sections logically together.

3.  **`TokenTextSplitter`:**
    * **Niche Use:** When you need to split text based on the **number of tokens** (which is how LLMs measure input size) rather than raw characters. This is useful for precise control over LLM input. It's especially handy when working directly with a tokenizer from an LLM.

---

**In summary:**

* Use `CharacterTextSplitter` for very structured data or when you absolutely must split only at specific characters.
* For everything else, especially with LLMs and RAG, `RecursiveCharacterTextSplitter` is usually your best friend.
* Use specialized splitters like `MarkdownHeaderTextSplitter` or `TokenTextSplitter` when your data has a specific format or when token-based splitting is critical.